In [4]:
import asyncio
import json
import time
from urllib.parse import quote
from bs4 import BeautifulSoup
from pyppeteer import launch
from pyppeteer.errors import PageError
#import gradio as gr
#from langdetect import detect
#from google_trans_new import google_translator

In [20]:
def scrape_post(url_or_shortcode: str) -> str:
    """Generate URL for fetching Instagram post data using GraphQL"""
    if "http" in url_or_shortcode:
        shortcode = url_or_shortcode.split("/p/")[-1].split("/")[0]
    else:
        shortcode = url_or_shortcode
    variables = {
        "shortcode": shortcode,
        "child_comment_count": 10000,
        "fetch_comment_count": 50,  # Adjust based on how many comments you want per request
        "parent_comment_count": 10000,
        "has_threaded_comments": True,
    }
    query_hash = "b3055c01b4b222b8a47dc12b090e4e64"
    doc_id = "18113378221181848"
    encoded_variables = quote(json.dumps(variables))
    url = f"https://www.instagram.com/graphql/query/?query_hash={query_hash}&variables={encoded_variables}"
    url = f"https://www.instagram.com/graphql/query/?doc_id={doc_id}&variables={encoded_variables}"
    
    return url

async def load_page(url):
    browser = await launch(options={'args': ['--no-sandbox', '--disable-setuid-sandbox']})
    page = await browser.newPage()
    await page.goto(url)
    await asyncio.sleep(3)  # Adjust as needed for page loading time
    content = await page.content()
    await browser.close()
    return content


def save_in_file(data, file_name):
    """Save data to a JSON file"""
    with open(file_name, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=2, ensure_ascii=False)

def extract_comments(parsed_data):
    """Extract comments from parsed JSON data"""
    comments = []
    try:
        edges = parsed_data['data']['shortcode_media']['edge_media_to_parent_comment']['edges']
        for edge in edges:
            comment = edge['node']['text'].strip()
            comments.append(comment)
            if 'edge_threaded_comments' in edge['node']:
                replies = edge['node']['edge_threaded_comments']['edges']
                for reply in replies:
                    reply_comment = reply['node']['text'].strip()
                    comments.append(reply_comment)
    except KeyError:
        print("Error: Unable to extract comments. Check if the data structure has changed.")
    return comments

async def scrape_comments(url):
    try:
        post_name = url.split('/')[-2]  # Extract post name from URL

        # Load all comments dynamically
        graphql_url = scrape_post(url)
        content = await load_page(graphql_url)

        # Parse content and extract comments
        parsed_data = json.loads(BeautifulSoup(content, "html.parser").find("pre").text)

        if parsed_data:
            comments = extract_comments(parsed_data)

            # Translate comments
            #translated_comments = await asyncio.gather(*[detect_and_translate(comment) for comment in comments])

            # Save comments to JSON with post name as key
            comments_data = {
                post_name: comments
            }
            save_in_file(comments_data, f'{post_name}_comments.json')

            return comments, f"Comments saved successfully in {post_name}_comments.json."
        else:
            return [], "Failed to load comments. Please check the URL or try again."
    except PageError as pe:
        return [], f"Error: {pe}"
    except Exception as e:
        return [], str(e)
'''
async def detect_and_translate(comment, target_lang="en"):
    """Detect language of the comment and translate it to target language if needed"""
    result_lang = detect(comment)

    if result_lang == target_lang or result_lang == 'en':
        return comment
    else:
        translator = google_translator()
        translated_text = await translator.translate(comment, lang_src=result_lang, lang_tgt=target_lang)
        return translated_text

def gradio_interface(url):
    translated_comments, message = asyncio.get_event_loop().run_until_complete(scrape_comments(url))
    return "\n".join(translated_comments), message

iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter Instagram post URL here..."),
    outputs=[gr.Textbox(lines=10, label="Translated Comments"), gr.Textbox(label="Status Message")],
    title="Instagram Comment Scraper with Translation",
    description="Enter the URL of an Instagram post to scrape and translate comments.",
)

# Para probar la interfaz de Gradio (esto bloqueará la ejecución en un entorno de Jupyter)
# iface.launch(debug=True, share=True)
'''

# Funciones para pruebas individuales (ejecutar en celdas separadas)


'''
async def test_detect_and_translate(text):
    translated = await detect_and_translate(text)
    print(f"Texto original: {text}")
    print(f"Texto traducido: {translated}")
'''
# Ejemplo de uso para pruebas (puedes descomentar y ejecutar en celdas separadas)
# instagram_url = "https://www.instagram.com/p/CwdT65hsj7G/"
# asyncio.run(test_scrape_comments(instagram_url))

# Ejemplo de prueba de traducción
# asyncio.run(test_detect_and_translate("Este es un comentario en español."))


'\nasync def test_detect_and_translate(text):\n    translated = await detect_and_translate(text)\n    print(f"Texto original: {text}")\n    print(f"Texto traducido: {translated}")\n'

In [27]:
await load_page(scrape_post(instagram_url))

e:\Users\1167486\AppData\Local\anaconda3\envs\Iris_WebScrapper\Lib\site-packages\pyppeteer\util.py:29: RuntimeWarning: coroutine 'run_scrape_test' was never awaited
  gc.collect()
[INFO] Starting Chromium download.


OSError: Chromium downloadable not found at https://storage.googleapis.com/chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip: Received <?xml version='1.0' encoding='UTF-8'?><Error><Code>NoSuchKey</Code><Message>The specified key does not exist.</Message><Details>No such object: chromium-browser-snapshots/Win_x64/1181205/chrome-win.zip</Details></Error>.


In [21]:
async def test_scrape_comments(url):
    comments, message = await scrape_comments(url)
    print("Mensaje:", message)
    if comments:
        print("Primeros 10 comentarios traducidos:")
        for i, comment in enumerate(comments[:10]):
            print(f"{i+1}. {comment}")
    else:
        print("No se encontraron comentarios o hubo un error.")

In [22]:
# Define una URL de Instagram para probar
instagram_url = "https://www.instagram.com/katyperry/reel/DJfNSmdRaLj/" # Reemplaza con una URL válida

In [23]:
# Prueba la función de scraping de comentarios
async def run_scrape_test(url):
    await test_scrape_comments(url)

# Ejecuta la prueba (esto bloqueará la celda hasta que termine)
#

In [24]:
async def run_scrape_test(url):
    await test_scrape_comments(url)


In [25]:
asyncio.run(run_scrape_test(instagram_url))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# Prueba la función de detección y traducción
async def run_translate_test(text):
    await test_detect_and_translate(text)

# Ejecuta la prueba de traducción
# asyncio.run(run_translate_test("Hola, ¿cómo estás?"))
# asyncio.run(run_translate_test("This is an English comment."))